# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🐊",
        "🐊",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
    ],
    [
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🌾",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🌲",
        "🌲",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🌾",
        "🌾",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🌾",
        "🌾",
        "🪨",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🌾",
        "🐊",
        "🌾",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🗻",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌲",
        "🌾",
        "🗻",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🌲",
        "🌲",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🌾",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🌾",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🪨",
        "🗻",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🌾",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🗻",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🗻",
        "🗻",
        "🗻",
        "🐊",
        "🐊",
        "🐊",
        "🐊",
    ],
    [
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
        "🌾",
        "🪨",
        "🪨",
        "🪨",
        "🌾",
        "🌾",
        "🌾",
    ],
]


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [2]:
test_world = [
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲"],
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲"],
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲"],
    ["🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌾"],
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [3]:
MOVES = [(0, -1), (1, 0), (0, 1), (-1, 0)]

## Costs

We can encode the costs described above in a `Dict`:

In [4]:
COSTS = {"🌾": 1, "🌲": 3, "🪨": 5, "🐊": 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (it does not return anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [5]:
import numpy as np
from typing import Tuple, List, Dict, Callable

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="heuristic"></a>
## heuristic

`heuristic` provides an estimate of the cost from the current position to the goal. It helps `a_star_search` determine which movement direction is likely the lowest cost overall cost. **Used by**: [a_star_search](#a_star_search)

* **current_location** Tuple[int, int]: the current location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the goal location, `(x, y)`.

**returns** int: The integer value of the estimate to the goal from the current location.

In [6]:
def heuristic(
    current_location: Tuple[int, int], goal: Tuple[int, int]
):  # you can add formal parameters
    h = np.abs(goal[0] - current_location[0]) + np.abs(goal[1] - current_location[1])
    return h

In [7]:
test_location = (0, 0)
test_goal = (5, 5)
a = heuristic(test_location, test_goal)
assert a == 10
test_location = (0, 0)
test_goal = (0, 0)
a = heuristic(test_location, test_goal)
assert a == 0
test_location = (5, 5)
test_goal = (5, 5)
a = heuristic(test_location, test_goal)
assert a == 0
test_location = (3, 3)
test_goal = (5, 5)
a = heuristic(test_location, test_goal)
assert a == 4

<a id="priority"></a>
## priority queue

`priority_queue` takes a list of tuples and sorts the list based on the first value in each element of the list. **Used by**: [a_star_search](#a_star_search)

* **queue** List[Tuple[int, Tuple[int, int]]]: the queue of tuples to be sorted

In [8]:
def priority_queue(
    queue: List[Tuple[int, Tuple[int, int]]],
):  # you can add formal parameters
    queue.sort(key=lambda test: test[0])
    pass

In [9]:
test = [(1, (3, 0)), (8, (2, 1)), (3, (1, 2)), (2, (0, 3))]
priority_queue(test)
assert test == [(1, (3, 0)), (2, (0, 3)), (3, (1, 2)), (8, (2, 1))]
test = [(1, (0, 3)), (8, (1, 2)), (3, (2, 1)), (2, (3, 0))]
priority_queue(test)
assert test == [(1, (0, 3)), (2, (3, 0)), (3, (2, 1)), (8, (1, 2))]
test = [(10, (0, 3)), (0, (1, 2)), (20, (2, 1)), (1, (3, 0))]
priority_queue(test)
assert test == [(0, (1, 2)), (1, (3, 0)), (10, (0, 3)), (20, (2, 1))]

<a id="a_star_search"></a>
## a_star_search

`a_star_search` attempts to find the shortest path from the starting posistion to the goal position by searching through a state space using defined legal moves to travesrse the defined world. `a_star_search` builds a graph of the space as it moves through the space and stores the cost to arrive at the current position, the current position, and the previous position. When `a_star_search` finds the goal, it reconstructs the shortest path by linking the child to their parent all the way to the starting position. **Uses**: [priority_queue](#priority_queue) and [heuristic](*heuristic) 

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model.


**returns** List[Tuple[int,Tuple[int, int],tuple[int,int]]]: the offsets needed to get from start state to the goal as a `List`. Includes [cost from start to current position, current position `(x,y)`, where moved from `(x,y)`]


In [10]:
def a_star_search(
    world: List[List[str]],
    start: Tuple[int, int],
    goal: Tuple[int, int],
    costs: Dict[str, int],
    moves: List[Tuple[int, int]],
    heuristic: Callable,
    priority_queue: Callable,
) -> List[Tuple[int, int]]:
    frontier = list()
    explored = list()
    children = list()
    g_n = 0
    hit = 0
    parent = (0, 0)
    path = list()

    heuristic_n = heuristic(start, goal)
    frontier.append((heuristic_n, g_n, start, parent))

    while frontier:
        current_state = frontier.pop(0)
        if current_state[2] == goal:
            explored.append((current_state[1], current_state[2], current_state[3]))
            traverse1 = 0
            while traverse1 <= (len(explored) - 2):
                for traverse2 in range(len(explored)):
                    if (
                        explored[len(explored) - traverse1 - 1][2]
                        == explored[len(explored) - traverse2 - 1][1]
                    ):
                        path.append(explored[len(explored) - traverse1 - 1])
                        traverse1 = traverse2
            return path

        for direction in moves:
            next_state = np.add(current_state[2], direction)
            if next_state[1] >= 0 and next_state[1] < len(world):
                if next_state[0] >= 0 and next_state[0] < len(world[0]):
                    if world[next_state[1]][next_state[0]] in costs:
                        children.append(
                            (direction, world[next_state[1]][next_state[0]])
                        )

        while children:
            child = children.pop()
            child_state = np.add(child[0], current_state[2])
            child_state = (child_state[0], child_state[1])
            for compare in range(len(frontier)):
                if child_state == frontier[compare][2]:
                    hit = 1
            for compare in range(len(explored)):
                if child_state == explored[compare][1]:
                    hit = 1
            if hit == 0:
                h_n = heuristic(child_state, goal)
                g_n = current_state[1] + costs[child[1]]
                f_n = g_n + h_n

                frontier.append((f_n, g_n, (child_state), current_state[2]))
                priority_queue(frontier)
            hit = 0
        explored.append((current_state[1], current_state[2], current_state[3]))

## pretty_print_solution

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.

**returns** int - The path cost.

In [11]:
def pretty_print_solution(
    world: List[List[str]],
    path: List[Tuple[int, Tuple[int, int]]],
) -> int:
    for traverse in range(len(path)):
        if (np.subtract(path[traverse][1], path[traverse][2]) == (0, 1)).all():
            world[path[traverse][1][1]][path[traverse][1][0]] = "⏬"
        elif (np.subtract(path[traverse][1], path[traverse][2]) == (0, -1)).all():
            world[path[traverse][1][1]][path[traverse][1][0]] = "⏫"
        elif (np.subtract(path[traverse][1], path[traverse][2]) == (1, 0)).all():
            world[path[traverse][1][1]][path[traverse][1][0]] = "⏩"
        elif (np.subtract(path[traverse][1], path[traverse][2]) == (-1, 0)).all():
            world[path[traverse][1][1]][path[traverse][1][0]] = "⏪"
    for display in range(len(world)):
        print("".join(world[display]))
    return path[0][0]

## Problems

## Problem 1. 

Execute `a_star_search` and `print_path` for the `test_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [12]:
test_start = (0, 0)
test_goal = (len(test_world[0]) - 1, len(test_world) - 1)
test_path = a_star_search(
    test_world, test_start, test_goal, COSTS, MOVES, heuristic, priority_queue
)
test_path_cost = pretty_print_solution(test_world, test_path)
print(f"total path cost: {test_path_cost}")
print(test_path)

🌾🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬⏩⏩⏩⏩⏩⏩
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
total path cost: 12
[(12, (6, 6), (6, 5)), (11, (6, 5), (6, 4)), (10, (6, 4), (6, 3)), (9, (6, 3), (5, 3)), (8, (5, 3), (4, 3)), (7, (4, 3), (3, 3)), (6, (3, 3), (2, 3)), (5, (2, 3), (1, 3)), (4, (1, 3), (0, 3)), (3, (0, 3), (0, 2)), (2, (0, 2), (0, 1)), (1, (0, 1), (0, 0))]


In [13]:
np.subtract(test_path[1][1], test_path[1][2])

array([0, 1])

## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [14]:
full_start = (10, 10)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(
    full_world, full_start, full_goal, COSTS, MOVES, heuristic, priority_queue
)
full_path_cost = pretty_print_solution(full_world, full_path)
print(f"total path cost: {full_path_cost}")
print(full_path)

🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🗻🗻🗻🗻🗻🗻🗻🌾🌾
🌾🌾🌾🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🪨🪨🪨🗻🗻🪨🪨
🌾🌾🌾🌾🪨🗻🗻🗻🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾🪨🪨🗻🗻🪨🌾
🌾🌾🌾🪨🪨🗻🗻🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾🪨🗻🗻🗻🪨🌾
🌾🪨🪨🪨🗻🗻🪨🪨🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾🪨🗻🪨🌾🌾
🌾🪨🪨🗻🗻🪨🪨🌾🌾🌾🌾🪨🗻🗻🗻🐊🐊🐊🌾🌾🌾🌾🌾🪨🌾🌾🌾
🌾🌾🪨🪨🪨🪨🪨🌾🌾🌾🌾🌾🌾🪨🗻🗻🗻🐊🐊🐊🌾🌾🪨🪨🪨🌾🌾
🌾🌾🌾🪨🪨🪨🌾🌾🌾🌾🌾🌾🪨🪨🗻🗻🌾🐊🐊🌾🌾🪨🪨🪨🌾🌾🌾
🌾🌾🌾🐊🐊🐊🌾🌾🪨🪨🪨🗻🗻🗻🗻🌾🌾🌾🐊🌾🪨🪨🪨🌾🌾🌾🌾
🌾🌾🐊🐊🐊🐊🐊⏪⏪⏪🗻🗻🗻🪨🌾🌾🌾🌾🌾🪨🗻🗻🗻🪨🌾🌾🌾
🌾🐊🐊🐊🐊🐊⏪⏬🪨🗻🗻🪨🌾🌾🌾🌾🐊🐊🌾🌾🪨🗻🗻🪨🌾🌾🌾
🐊🐊🐊🐊🐊🌾⏬🪨🪨🗻🗻🪨🌾🐊🐊🐊🐊🌾🌾🌾🪨🗻🪨🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾⏬🪨🌲🌲🪨🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾🪨🌾🌾🌾🌾🌾
🌾🌾🌾🌾🗻🌾⏬🌲🌲🌲🌲🪨🪨🪨🪨🌾🐊🐊🐊🌾🌾🪨🗻🪨🌾🌾🌾
🌾🌾🌾🗻🗻🗻⏬🌲🌲🌲🌲🌲🗻🗻🗻🪨🪨🌾🐊🌾🪨🗻🗻🪨🌾🌾🌾
🌾🌾🗻🗻🌲🌲⏬🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🌾🌾🗻🗻🗻🌾🌾🌾🌾🌾
🌾🌾🌾🗻🗻🌲⏬🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🗻🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🗻🗻🗻⏬⏩🌲🌲🌲🌲🌲🌲🌾🌾🌾🪨🪨🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🗻🗻🗻⏬⏩⏩⏩⏩⏩⏩⏩⏩🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾🌾🪨🪨🪨🪨🗻🗻🌲🌲🌲🌲🌲🌾🗻⏬🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🪨🪨🪨🗻🗻🗻🌲🌲🗻🗻🌾⏬⏩⏩⏩⏩🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾🪨🪨🪨🗻🗻🗻🗻🌾🌾🌾🌾🪨🪨⏬⏩🐊🐊🐊🐊🐊🐊
🌾🪨🪨🌾🌾🪨🪨🪨🪨🪨🌾🌾🌾🌾🌾🪨🪨🗻🗻🪨⏬⏩🐊🐊🐊🐊🐊
🪨🗻🪨🪨🪨🪨🌾🌾🌾🌾🌾🗻🗻🗻🪨🪨🗻🗻🌾🗻🗻⏬⏩⏩🐊🐊🐊
🪨🗻🗻🗻🪨🌾🌾🌾🌾🌾🪨🪨🗻🗻🗻🗻🪨🪨🪨🪨🗻🗻🗻⏬🐊🐊🐊
🪨🪨🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🪨🪨🪨🪨🪨🌾🌾🌾🌾🪨🪨⏬⏩⏩⏩
total path cost: 98
[(98, (26, 26), (25, 26)), (97, (25, 26), (24, 26)), (96, (24, 26), (23, 26)), (95, (23, 26), (23, 25)), (90, (23, 25), (23, 24)), (83, (23, 24), (22, 24)), (76, (22, 24), (21, 24)), (71, (21, 24), (21, 23)), (66, (21, 23), 